In [1]:
# Importing libraries
import psycopg2
import pandas as pd

In [2]:
def create_database():
    '''
    Creating new database for PostgreSQL
    '''
    # Connecting to default database
    conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=pass")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    
    # Creating the database
    cur.execute("DROP DATABASE IF EXISTS accounts")
    cur.execute("CREATE DATABASE accounts")
    
    # Close connection to default database
    conn.close()
    
    # Connecting to the newly made database
    conn = psycopg2.connect("host=localhost dbname=accounts user=postgres password=pass")
    cur = conn.cursor()
    
    return cur, conn

In [3]:
# Reading data
accountcountry = pd.read_csv('Wealth-AccountsCountry.csv')

In [5]:
# Shortening data
accountcountry = accountcountry[['Code', 'Short Name', 'Table Name', 'Long Name', 'Currency Unit']]

In [22]:
# Using the function to make the database
cur, conn = create_database()

In [23]:
# Making the new table in database
accountscountry_table = ("""
CREATE TABLE IF NOT EXISTS accountscountry(
country_code VARCHAR PRIMARY KEY,
short_name VARCHAR,
table_name VARCHAR,
long_name VARCHAR,
currency_unit VARCHAR
)
""")

In [24]:
# Executing the query
cur.execute(accountscountry_table)
conn.commit()

In [9]:
conn.set_session(autocommit=True)

In [10]:
# Making the new table in database
accountsdata_table = ("""
CREATE TABLE IF NOT EXISTS accountsdata(
country_name VARCHAR,
country_code VARCHAR,
series_name VARCHAR,
series_code VARCHAR,
year_1995 VARCHAR,
year_2000 VARCHAR,
year_2005 VARCHAR,
year_2010 VARCHAR
)
""")

In [11]:
# Executing the query
cur.execute(accountsdata_table)
conn.commit()

In [22]:
# Adding data in the table
account_country_insert = ("""
INSERT INTO accountscountry(
country_code,
short_name,
table_name,
long_name,
currency_unit)
VALUES (%s, %s, %s, %s, %s)
""")

In [27]:
# Removing the key
cur.execute("""
ALTER TABLE accountscountry
DROP CONSTRAINT accountscountry_pkey
""")

In [28]:
# Getting the data
for i, row in accountcountry.iterrows():
    cur.execute(account_country_insert, list(row))

In [39]:
# Closing
cur.close()
conn.close()